In [ ]:
file_name_txt = "data/5_unten_fahrt.txt";
file_name_txt = "data/unten_5_7_8_9_4.txt";

In [ ]:
using Base64
function base16to64(st::AbstractString)
    bytes = hex2bytes(st)
    return bytes
end

val = base16to64("EA")
typeof(val[1])

In [ ]:
# Load file status
function show_file_status(name=String)
    file_status = stat(name)
    println("File: ", name, " has ", file_status.size, " Bytes")
end

function split_symbol_string(symbol=String)
    _, right = split(symbol, "RX:")
    # _, right = split(symbol, "TX:")
    _, right = split(right, " ")
    return right
end

show_file_status(file_name_txt)

In [ ]:
function load_uart_array(name=String)
    string_array = readlines(name)
    exclude_strings = ["Start", "Stop"]
    start_symbol = "D0"
    n_symbols_per_msg = 6
    
    uart_array = Array{UInt8}(undef, 1, n_symbols_per_msg)
    
    col_idx = 1
    row_idx = 1
    for str in string_array
        if occursin(exclude_strings[1], str) || occursin(exclude_strings[2], str)
            continue
        else
            symbol = split_symbol_string(str)
            if col_idx <= n_symbols_per_msg
                uart_array[row_idx, col_idx] = base16to64(symbol)[1]
            else
                uart_array = vcat(uart_array, Array{UInt8}(undef, 1, n_symbols_per_msg))
                row_idx += 1
                col_idx = 1
                uart_array[row_idx, col_idx] = base16to64(symbol)[1]
            end
            col_idx += 1
        end
    end

    return uart_array
end

uart_array = load_uart_array(file_name_txt);

In [ ]:
function show_array_props(array=Array{UInt8})
    array_length = size(array)[1]
    n_messages = size(array)[2]
    println("Array length: ", array_length)
    println("Message length: ", n_messages)
    
    unique_symbols = unique(array)
    n_unique_symbols = length(unique_symbols)
    println("Number of unique symbols: ", n_unique_symbols)
    
    # Iterate columns
    for idx in 1:n_messages
        unique_column = unique(array[:, idx])
        println(idx, ".Byte: ", unique_column)
    end
    
    # Iterate data in message
    frame_array = Array{Array{UInt8}}(undef, array_length, 1)
    idx = 1
    for frame in eachrow(array)
        frame_array[idx] = frame[3:6]
        idx = idx + 1
    end

    unique_messages = unique(frame_array)
    println("Number of individual data frames: ", length(unique_messages))
    
end

show_array_props(uart_array)